In [1]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 3.3 MB 32.3 MB/s 
     |████████████████████████████████| 895 kB 65.4 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████| 61 kB 488 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/678M [00:00<?, ?B/s]

In [6]:
import time
import torch
from sklearn.metrics.pairwise import cosine_similarity

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


pairwise_sentences_eliza = [
    "Я пошел гулять",
    "вы пошел гулять"
]
pairwise_sentences_seq2seq = [
    "ы давайте наконец unk",
    "Я пошел гулять"
]
pairwise_sentences_rut5 = [
    "Я пошел гулять",
    "Я знаю, что ты не можешь пойти вместе?"
]
pairwise_sentences_rugpt = [
    "Я пошел гулять",
    "Вы не можете пойти погулять, потому что вы в данный момент находитесь на работе и"
]

list_paiwise_sents = [pairwise_sentences_eliza,pairwise_sentences_seq2seq,pairwise_sentences_rut5,pairwise_sentences_rugpt]
for item in list_paiwise_sents:
  print(item)
  time_start_similarity = time.time()
  encoded_input = tokenizer(item, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
      model_output = model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = sentence_embeddings.detach().numpy()
  cosine_array = cosine_similarity(
        [sentence_embeddings[0]],
        sentence_embeddings[1:]
    )

  print(cosine_array.sum()/(2-1))
  print("Similarity Calculation time: ", time.time() - time_start_similarity)


['Я пошел гулять', 'вы пошел гулять']
0.9797183275222778
Similarity Calculation time:  0.28054213523864746
['ы давайте наконец unk', 'Я пошел гулять']
0.45563235878944397
Similarity Calculation time:  0.27045106887817383
['Я пошел гулять', 'Я знаю, что ты не можешь пойти вместе?']
0.46235471963882446
Similarity Calculation time:  0.3090529441833496
['Я пошел гулять', 'Вы не можете пойти погулять, потому что вы в данный момент находитесь на работе и']
0.43418213725090027
Similarity Calculation time:  0.5418951511383057


In [7]:

pairwise_sentences_eliza = [
    "Я чувствую обиду и злость за свой поступок",
    "Когда вы обычно чувствуете обиду и злость за свой?"
]
pairwise_sentences_seq2seq = [
    "Я чувствую обиду и злость за свой поступок",
    "unk  unk у обвинения или больше unk unk unk с unk unk"
]
pairwise_sentences_rut5 = [
    "Я чувствую обиду и злость за свой поступок",
    "Что это за поступок?" 
]
pairwise_sentences_rugpt = [
    "Я чувствую обиду и злость за свой поступок",
    "Здравствуйте, уважаемый Алексей! Спасибо Вам большое от всей души!!! Мы обязательно исправим Вашу ошибку"
]

list_paiwise_sents = [pairwise_sentences_eliza,pairwise_sentences_seq2seq,pairwise_sentences_rut5,pairwise_sentences_rugpt]
for item in list_paiwise_sents:
  print(item)
  time_start_similarity = time.time()
  encoded_input = tokenizer(item, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
      model_output = model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = sentence_embeddings.detach().numpy()
  cosine_array = cosine_similarity(
        [sentence_embeddings[0]],
        sentence_embeddings[1:]
    )

  print(cosine_array.sum()/(2-1))
  print("Similarity Calculation time: ", time.time() - time_start_similarity)


['Я чувствую обиду и злость за свой поступок', 'Когда вы обычно чувствуете обиду и злость за свой?']
0.915865957736969
Similarity Calculation time:  0.16571331024169922
['Я чувствую обиду и злость за свой поступок', 'unk  unk у обвинения или больше unk unk unk с unk unk']
0.5969514846801758
Similarity Calculation time:  0.21097636222839355
['Я чувствую обиду и злость за свой поступок', 'Что это за поступок?']
0.7133703231811523
Similarity Calculation time:  0.15615463256835938
['Я чувствую обиду и злость за свой поступок', 'Здравствуйте, уважаемый Алексей! Спасибо Вам большое от всей души!!! Мы обязательно исправим Вашу ошибку']
0.575518786907196
Similarity Calculation time:  0.22075796127319336


In [8]:

pairwise_sentences_eliza = [
    "Снег автомату рознь, а собака скользкая",
    "Как ты себя чувствуешь?"
]
pairwise_sentences_seq2seq = [
    "Снег автомату рознь, а собака скользкая",
    "unk приготовьтесь на минутку ли"
]
pairwise_sentences_rut5 = [
    "Снег автомату рознь, а собака скользкая",
    "О, ну что ж. Какая собака?"
]
pairwise_sentences_rugpt = [
    "Снег автомату рознь, а собака скользкая",
    "Да нет же! У меня была такая проблема с собакой - она не могла пройти"
]

list_paiwise_sents = [pairwise_sentences_eliza,pairwise_sentences_seq2seq,pairwise_sentences_rut5,pairwise_sentences_rugpt]
for item in list_paiwise_sents:
  print(item)
  time_start_similarity = time.time()
  encoded_input = tokenizer(item, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
      model_output = model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = sentence_embeddings.detach().numpy()
  cosine_array = cosine_similarity(
        [sentence_embeddings[0]],
        sentence_embeddings[1:]
    )

  print(cosine_array.sum()/(2-1))
  print("Similarity Calculation time: ", time.time() - time_start_similarity)


['Снег автомату рознь, а собака скользкая', 'Как ты себя чувствуешь?']
0.38018471002578735
Similarity Calculation time:  0.15279221534729004
['Снег автомату рознь, а собака скользкая', 'unk приготовьтесь на минутку ли']
0.49534058570861816
Similarity Calculation time:  0.15686869621276855
['Снег автомату рознь, а собака скользкая', 'О, ну что ж. Какая собака?']
0.5749257802963257
Similarity Calculation time:  0.15490317344665527
['Снег автомату рознь, а собака скользкая', 'Да нет же! У меня была такая проблема с собакой - она не могла пройти']
0.5466763973236084
Similarity Calculation time:  0.20959854125976562


In [9]:

pairwise_sentences_eliza = [
    "Скажи мне что-либо приятное",
    "Как ты себя чувствуешь?"
]
pairwise_sentences_seq2seq = [
    "Скажи мне что-либо приятное",
    "я unk со всеми дэнни но вы unk прямо без своей"
]
pairwise_sentences_rut5 = [
    "Скажи мне что-либо приятное",
    "Что это?"
]
pairwise_sentences_rugpt = [
    "Скажи мне что-либо приятное",
    "Я хочу, чтобы ты была счастлива и довольна своей жизнью! Ты должна быть счастливой в настоящем моменте"
]

list_paiwise_sents = [pairwise_sentences_eliza,pairwise_sentences_seq2seq,pairwise_sentences_rut5,pairwise_sentences_rugpt]
for item in list_paiwise_sents:
  print(item)
  time_start_similarity = time.time()
  encoded_input = tokenizer(item, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
      model_output = model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  sentence_embeddings = sentence_embeddings.detach().numpy()
  cosine_array = cosine_similarity(
        [sentence_embeddings[0]],
        sentence_embeddings[1:]
    )

  print(cosine_array.sum()/(2-1))
  print("Similarity Calculation time: ", time.time() - time_start_similarity)


['Скажи мне что-либо приятное', 'Как ты себя чувствуешь?']
0.7708983421325684
Similarity Calculation time:  0.153548002243042
['Скажи мне что-либо приятное', 'я unk со всеми дэнни но вы unk прямо без своей']
0.39786145091056824
Similarity Calculation time:  0.1918497085571289
['Скажи мне что-либо приятное', 'Что это?']
0.5968673229217529
Similarity Calculation time:  0.1268160343170166
['Скажи мне что-либо приятное', 'Я хочу, чтобы ты была счастлива и довольна своей жизнью! Ты должна быть счастливой в настоящем моменте']
0.750847578048706
Similarity Calculation time:  0.19956207275390625
